x0 is co course,x1 is c1 course , x2 is c2 course, x3 is c3 course ,x4  is course c4, x5 is course c5, x6 iscourse c6, x7 course c7,  x8 course is x8,x9 is course c9
 max   4x0+3x1+6x2+5x3+2x4+6x5+4x6+8x7+8x8+6x9
 x0+x1+x2+x3+x4+x5+x6+x7+x8+x9>=26,
x0+x1+x2+x3+x4+x5+x6+x7+x8+x9<=8,
x1+x7<=1,
x1+x9<=1,
x2+x8<=1,
x4+x5<=2,
x0+x2+x1++x3>=1
x0+x2+x1++x3<=2,
x5+x9<=1,
x6+x8+x9<=2,




In [234]:
!pip install  -q pyomo

In [235]:
from pyomo.environ import *

In [236]:
import numpy as np

In [237]:
model=ConcreteModel()


In [238]:
M=11
N=10


In [239]:
row_indices=np.arange(M)
print(row_indices)

[ 0  1  2  3  4  5  6  7  8  9 10]


In [240]:
obj_coef_c=np.array([4,3,6,5,2,6,4,8,8,6])
constr_coef=np.array([[-4,-3,-6,-5,-2,-6,-4,-8,-8,-6],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[1,1,1,1,1,1,1,1,1,1],[0,0,1,0,0,0,0,1,0,0],[0,1,0,0,0,0,0,0,0,1],[0,0,1,0,0,0,0,0,1,0],[0,0,0,1,1,0,0,0,0,0],[-1,-1,-1,-1,0,0,0,0,0,0],[1,1,1,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,1],[0,0,0,0,0,0,1,0,1,1]])
constr_rhs_b=([-26,-4,8,1,1,1,2,-1,2,1,2,])


In [241]:
col_indices = np.arange(N)
print(col_indices)

[0 1 2 3 4 5 6 7 8 9]


In [242]:
model.x = Var(col_indices,domain=Binary)



In [243]:
model.constraints = ConstraintList()

In [244]:
for i in row_indices:
  model.constraints.add(sum(constr_coef[i][j]*model.x[j] for j in col_indices) <= constr_rhs_b[i])

In [245]:
model.objective = Objective(expr =summation(obj_coef_c,model.x), sense=maximize)
model.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :   

In [246]:
!apt-get install -y -qq coinor-cbc

In [247]:

opt_cbc = SolverFactory('cbc')

In [248]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)



Problem: 
- Name: unknown
  Lower bound: 37.0
  Upper bound: 37.0
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 10
  Number of binary variables: 10
  Number of integer variables: 10
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.023654937744140625
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.023654937744140625


In [249]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x'+str(i+0)+'= ', model.x[i].value)
 
 
print('\nConstraints')
model.constraints.display()


Objective =  37.0

Decision Variables
x0=  1.0
x1=  0.0
x2=  0.0
x3=  1.0
x4=  1.0
x5=  1.0
x6=  1.0
x7=  1.0
x8=  1.0
x9=  0.0

Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -37.0 : -26.0
      2 :  None :  -7.0 :  -4.0
      3 :  None :   7.0 :   8.0
      4 :  None :   1.0 :   1.0
      5 :  None :   0.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   2.0 :   2.0
      8 :  None :  -2.0 :  -1.0
      9 :  None :   2.0 :   2.0
     10 :  None :   1.0 :   1.0
     11 :  None :   2.0 :   2.0


3. [R] Solver your MILP using cbc solver and report the optimal values of the variables and the objective function value.



the maximam vaule =  37.0


Decision Variables
x0=  1.0
x1=  0.0
x2=  0.0
x3=  1.0
x4=  1.0
x5=  1.0
x6=  1.0
x7=  1.0
x8=  1.0
x9=  0.0




#4
 domain is reals

In [250]:
model1=ConcreteModel()


In [251]:
M=11
N=10


In [252]:
row_indices=np.arange(M)
print(row_indices)

[ 0  1  2  3  4  5  6  7  8  9 10]


In [253]:
obj_coef_c=np.array([4,3,6,5,2,6,4,8,8,6])
constr_coef=np.array([[-4,-3,-6,-5,-2,-6,-4,-8,-8,-6],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[1,1,1,1,1,1,1,1,1,1],[0,0,1,0,0,0,0,1,0,0],[0,1,0,0,0,0,0,0,0,1],[0,0,1,0,0,0,0,0,1,0],[0,0,0,1,1,0,0,0,0,0],[-1,-1,-1,-1,0,0,0,0,0,0],[1,1,1,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,1],[0,0,0,0,0,0,1,0,1,1]])
constr_rhs_b=([-26,-4,8,1,1,1,2,-1,2,1,2,])
lb=np.array([0,0,0,0,0,0,0,0,0,0])
ub=np.array([1,1,1,1,1,1,1,1,1,1]) 



In [254]:
col_indices = np.arange(N)
print(col_indices)

[0 1 2 3 4 5 6 7 8 9]


In [255]:
model1.x = Var(col_indices,domain=Reals)


In [256]:
for i in col_indices:
  model1.x[i].setlb(lb[i])
  model1.x[i].setub(ub[i])


In [257]:
model1.constraints = ConstraintList()

In [258]:
for i in row_indices:
  model1.constraints.add(sum(constr_coef[i][j]*model1.x[j] for j in col_indices) <= constr_rhs_b[i])

In [259]:
model1.objective = Objective(expr =summation(obj_coef_c,model1.x), sense=maximize)
model1.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 :  None :   

In [260]:
!apt-get install -y -qq coinor-cbc

In [261]:

opt_cbc = SolverFactory('cbc')

In [262]:
result = opt_cbc.solve(model1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)



Problem: 
- Name: unknown
  Lower bound: 37.0
  Upper bound: 37.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 11
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 3
  Error rc: 0
  Time: 0.04980158805847168
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.04980158805847168


In [263]:
# display solution
print('\nObjective = ', model1.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x'+str(i+0)+'= ', model1.x[i].value)
 
 
print('\nConstraints')
model1.constraints.display()



Objective =  37.0

Decision Variables
x0=  1.0
x1=  0.0
x2=  0.0
x3=  1.0
x4=  1.0
x5=  1.0
x6=  1.0
x7=  1.0
x8=  1.0
x9=  0.0

Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -37.0 : -26.0
      2 :  None :  -7.0 :  -4.0
      3 :  None :   7.0 :   8.0
      4 :  None :   1.0 :   1.0
      5 :  None :   0.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   2.0 :   2.0
      8 :  None :  -2.0 :  -1.0
      9 :  None :   2.0 :   2.0
     10 :  None :   1.0 :   1.0
     11 :  None :   2.0 :   2.0


there is no chnages of optimal value when used binary and when used reals domain
in this qustion 



5 ans-if the constrint is satifys that is not meaning  is optimal sulution  exits if may or may not be optimal solution

---







#6 qustion new objetive function

max   4x0+3x1+6x2+5x3+2x4+6x5+4x6+8x7+8x8+6x9
new add constriant 
x10+x5<=1,
x10+x6<=1



In [264]:
model.objective.deactivate()
M = 12
model.x10= Var(domain=Binary) 
model.constraints.add(expr = model.x10+ model.x[5] <=1)
model.constraints.add(expr = model.x10+ model.x[6] <=1)

In [265]:
model.new_objective = Objective(expr =12* model.x10 + summation(obj_coef_c,model.x) ,sense=maximize)

In [266]:
model.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   13 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

2 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False :  True : Binary
          1 :     0 :   0.0 :     1 : False :  True : Binary
          2 :     0 :   0.0 :     1 : False :  True : Binary
          3 :     0 :   1.0 :     1 : False :  True : Binary
          4 :     0 :   1.0 :     1 : False :  True : Binary
          5 :     0 :   1.0 :     1 : False :  True : Binary
          6 :     0 :   1.0 :     1 : False :  True : Binary
          7 :     0 :   1.0 :     1 : False :  True : Binary
          8 :     0 :   

In [267]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)



Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.04939007759094238
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.04939007759094238


In [268]:

# display solution
print('\nObjective = ', model.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x'+str(i+0)+'= ', model.x[i].value)
 
print('x10 = ', model.x10.value)  

print('\nConstraints')
model.constraints.display()


Objective =  45.0

Decision Variables
x0=  1.0
x1=  0.0
x2=  0.0
x3=  1.0
x4=  1.0
x5=  0.0
x6=  0.0
x7=  1.0
x8=  1.0
x9=  1.0
x10 =  1.0

Constraints
constraints : Size=13
    Key : Lower : Body  : Upper
      1 :  None : -33.0 : -26.0
      2 :  None :  -6.0 :  -4.0
      3 :  None :   6.0 :   8.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   2.0 :   2.0
      8 :  None :  -2.0 :  -1.0
      9 :  None :   2.0 :   2.0
     10 :  None :   1.0 :   1.0
     11 :  None :   2.0 :   2.0
     12 :  None :   1.0 :   1.0
     13 :  None :   1.0 :   1.0


 7 ans_*the* maximum value are=  45.0

Decision Variables
x0=  1.0
x1=  0.0
x2=  0.0
x3=  1.0
x4=  1.0
x5=  0.0
x6=  0.0
x7=  1.0
x8=  1.0
x9=  1.0
x10 =  **1.0**

In [269]:
model1.objective.deactivate()
M = 12
model1.x10= Var(domain=Reals) 
model1.constraints.add(expr = model1.x10+ model1.x[5] <=1)
model1.constraints.add(expr = model1.x10+ model1.x[6] <=1)

In [270]:
model1.new_objective = Objective(expr =12* model1.x10 + summation(obj_coef_c,model1.x) ,sense=maximize)

In [271]:
model1.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   13 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

2 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False :  True :  Reals
          1 :     0 :   0.0 :     1 : False :  True :  Reals
          2 :     0 :   0.0 :     1 : False :  True :  Reals
          3 :     0 :   1.0 :     1 : False :  True :  Reals
          4 :     0 :   1.0 :     1 : False :  True :  Reals
          5 :     0 :   1.0 :     1 : False :  True :  Reals
          6 :     0 :   1.0 :     1 : False :  True :  Reals
          7 :     0 :   1.0 :     1 : False :  True :  Reals
          8 :     0 :   

In [272]:
result = opt_cbc.solve(model1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)



Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 14
  Number of variables: 12
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.04853034019470215
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.04853034019470215


In [273]:

# display solution
print('\nObjective = ', model1.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x'+str(i+0)+'= ', model1.x[i].value)
 
print('x10 = ', model1.x10.value)  

print('\nConstraints')
model.constraints.display()


Objective =  45.0

Decision Variables
x0=  1.0
x1=  0.0
x2=  0.0
x3=  1.0
x4=  1.0
x5=  0.0
x6=  0.0
x7=  1.0
x8=  1.0
x9=  1.0
x10 =  1.0

Constraints
constraints : Size=13
    Key : Lower : Body  : Upper
      1 :  None : -33.0 : -26.0
      2 :  None :  -6.0 :  -4.0
      3 :  None :   6.0 :   8.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   2.0 :   2.0
      8 :  None :  -2.0 :  -1.0
      9 :  None :   2.0 :   2.0
     10 :  None :   1.0 :   1.0
     11 :  None :   2.0 :   2.0
     12 :  None :   1.0 :   1.0
     13 :  None :   1.0 :   1.0


in this case this qustion ans are same but in genral the  will not same 